# 指令微调

In [1]:
"""
The main program for finetuning LLMs with Huggingface Transformers Library.

ALL SECTIONS WHERE CODE POSSIBLY NEEDS TO BE FILLED IN ARE MARKED AS TODO.
"""

import argparse
from dataclasses import dataclass, field
from typing import Optional, List, Dict
import sys
import torch
from transformers import TrainingArguments, HfArgumentParser, Trainer, AutoTokenizer, AutoModelForCausalLM
import datasets
from peft import LoraConfig, TaskType, get_peft_model
print("done")

e:\Anaconda\envs\dp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


done


In [2]:
# Define the arguments required for the main program.
# NOTE: You can customize any arguments you need to pass in.
@dataclass
class ModelArguments:
    """Arguments for model
    """
    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The path to the LLM to fine-tune or its name on the Hugging Face Hub."
        }
    )
    torch_dtype: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "Override the default `torch.dtype` and load the model under this dtype."
            ),
            "choices": ["bfloat16", "float16", "float32"],
        },
    )
    # TODO: add your model arguments here
    pass


@dataclass
class DataArguments:
    """Arguments for data
    """
    dataset_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The path to the fine-tuning dataset or its name on the Hugging Face Hub."
        }
    )
    # TODO: add your data arguments here
    
@dataclass
class TrainingArguments(TrainingArguments):
    """Arguments for training
    """
    output_dir: str = field(
        default=None
    )
    per_device_train_batch_size: Optional[int] = field(
        default=1
    )
    save_strategy: Optional[str] = field(
        default="steps"
    )
    save_steps: Optional[int] = field(
        default=10000
    )
    report_to: Optional[str] = field(
        default="tensorboard"
    )
    warmup_ratio: Optional[float] = field(
        default=0.03
    )
    logging_steps: Optional[int] = field(
        default=300
    )
    num_train_epochs: Optional[int] = field(
        default=1
    )
    learning_rate: Optional[float] = field(
        default=1e-5
    )
    optim: Optional[str] = field(
        default="adamw_hf"
    )
    lr_scheduler_type: Optional[str] = field(
        default="cosine"
    )
    weight_decay: Optional[float] = field(
        default=0.003
    )
    print("done")

done


In [8]:
# The main function
# NOTE You can customize some logs to monitor your program.
def finetune():
    
    config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        inference_mode=False,
        r=16,
        lora_alpha=32,
        lora_dropout=0.1
    )
    # TODO Step 1: Define an arguments parser and parse the arguments
    # NOTE Three parts: model arguments, data arguments, and training arguments
    # HINT: Refer to 
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/internal/trainer_utils#transformers.HfArgumentParser
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/main_classes/trainer#transformers.TrainingArguments
    parser = HfArgumentParser((ModelArguments, DataArguments, TrainingArguments))
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()
    print("step1 done")
    print(training_args)

    # TODO Step 2: Load tokenizer and model
    # HINT 1: Refer to
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/main_classes/tokenizer#tokenizer
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/model_doc/qwen2
    # HINT 2: To save training GPU memory, you need to set the model's parameter precision to half-precision (float16 or bfloat16).
    #         You may also check other strategies to save the memory!
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/model_doc/llama2#usage-tips
    #   * https://huggingface.co/docs/transformers/perf_train_gpu_one
    #   * https://www.53ai.com/news/qianyanjishu/2024052494875.html
    tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path)
    model = AutoModelForCausalLM.from_pretrained(model_args.model_name_or_path, torch_dtype=model_args.torch_dtype, device_map="cuda:0")
    model = get_peft_model(model, config)
    print("step2 done")
    
    
    # TODO Step 3: Load dataset
    # HINT: https://huggingface.co/docs/datasets/v3.1.0/en/package_reference/main_classes#datasets.Dataset
    dataset = datasets.load_dataset(data_args.dataset_path)
    dataset = dataset["train"]
    print("step3 done")
    
    # TODO Step 4: Define the data collator function
    # NOTE During training, for each model parameter update, we fetch a batch of data, perform a forward and backward pass,
    # and then update the model parameters. The role of the data collator is to process the data (e.g., padding the data within
    # a batch to the same length) and format the batch into the input required by the model.
    #
    # In this assignment, the purpose of the custom data_collator is to process each batch of data from the dataset loaded in
    # Step 3 into the format required by the model. This includes tasks such as tokenizing the data, converting each token into 
    # an ID sequence, applying padding, and preparing labels.
    # 
    # HINT:
    #   * Before implementation, you should:
    #      1. Clearly understand the format of each sample in the dataset loaded in Step 3.
    #      2. Understand the input format required by the model (https://huggingface.co/docs/transformers/model_doc/qwen2#transformers.Qwen2ForCausalLM).
    #         Reading its source code also helps!
    padding = -100
    def preprocess(dataset, max_len=1024):
        question_template = '<|im_start|>system\nYou are a knowledgeable assistant. Please give a detailed response to the following question.<|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n'
        prompt_template = '<|im_start|>system\nYou are a knowledgeable assistant. Please give a detailed response to the following question.<|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n{response}<|im_end|>' #定义训练时用的prompt
        model_inputs = {'input_ids': [], 'attention_mask':[], 'labels': []} #初始化返回的字典
        for i in range(len(dataset['instruction'])): #遍历
            instruct = dataset['instruction'][i] if dataset['instruction'][i] else '' # 有些数据的instruct和input没有内容，为None
            input = dataset['input'][i] if dataset['input'][i] else ''
            question = instruct+ '\n' + input
            prompt = question_template.format(
                        question=question,
                        #response=dataset['output'][i],
                    ) #获得prompt
            instruction = tokenizer(prompt,add_special_tokens=False,)
            response = tokenizer(dataset['output'][i],add_special_tokens=False)
            input_ids = instruction['input_ids']+response['input_ids']+[tokenizer.pad_token_id]
            attention_mask = (instruction["attention_mask"] + response["attention_mask"] + [1])
            labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]

            if len(input_ids) > max_len:
                input_ids = input_ids[:max_len]
                attention_mask = attention_mask[:max_len]
                labels = labels[:max_len]
            model_inputs['input_ids'].append(input_ids)
            model_inputs['attention_mask'].append(attention_mask)
            model_inputs['labels'].append(labels)
        return model_inputs
    
    dataset = dataset.map(preprocess, batched=True, num_proc=1) #用提前对dataset进行map操作替代custom data_collator，节约后续训练时间。
    print("step4 done")
    
    def data_collator(batch: List[Dict]):
        """
        batch: list of dict, each dict of the list is a sample in the dataset.
        """
        pass
    
    
    
    # TODO Step 5: Define the Trainer
    # HINT: https://huggingface.co/docs/transformers/main_classes/trainer
    trainer = Trainer(
        train_dataset=dataset,
        #eval_dataset=dataset["test"], 不进行训练测试集切分，而使用后续的数据集进行评估
        tokenizer=tokenizer,
        args=training_args,
        model=model,
    )
    print("step5 done")
    # Step 6: Train!
    trainer.train()
print("done")

done


In [9]:
# Pass your training arguments.
# NOTE [IMPORTANT!!!] DO NOT FORGET TO PASS PROPER ARGUMENTS TO SAVE YOUR CHECKPOINTS!!!
sys.argv = [
    "notebook", 
    "--model_name_or_path", "E:/Homework/NLP/NLP_Project_2/Qwen2.5-1.5B",
    "--torch_dtype", "bfloat16", #使用bfloat16来减小显存需求
    "--dataset_path", "E:/Homework/NLP/NLP_Project_2/alpaca-cleaned",
    "--output_dir", "E:/Homework/NLP/NLP_Project_2/output",
    "--per_device_train_batch_size", "1", #16G显存使用4会爆显
    "--save_strategy", "steps",
    "--save_steps", "10000",
    "--report_to", "tensorboard",
    "--warmup_ratio", "0.03",
    "--logging_steps", "100",
    "--num_train_epochs", "1",
    "--learning_rate", "1e-5",
    "--optim", "adamw_hf",
    "--weight_decay", "1e-3",
    "--lr_scheduler_type", "cosine",
]
finetune()

step1 done
TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.

C:\Users\86150\AppData\Local\Temp\ipykernel_20332\2547301790.py:99: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
e:\Anaconda\envs\dp\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 100/51760 [00:26<3:33:38,  4.03it/s]

{'loss': 1.4311, 'grad_norm': 7.603107929229736, 'learning_rate': 6.439150032195751e-07, 'epoch': 0.0}


  0%|          | 200/51760 [00:51<3:31:39,  4.06it/s]

{'loss': 1.268, 'grad_norm': 1.0809446573257446, 'learning_rate': 1.2878300064391501e-06, 'epoch': 0.0}


...

{'loss': 0.9367, 'grad_norm': 2.332582473754883, 'learning_rate': 2.505806595382021e-10, 'epoch': 1.0}


100%|█████████▉| 51700/51760 [3:45:02<00:14,  4.08it/s]

{'loss': 1.0188, 'grad_norm': 3.8935835361480713, 'learning_rate': 3.523815819195253e-11, 'epoch': 1.0}


100%|██████████| 51760/51760 [3:45:18<00:00,  3.83it/s]

{'train_runtime': 13518.0159, 'train_samples_per_second': 3.829, 'train_steps_per_second': 3.829, 'train_loss': 1.0298691117229197, 'epoch': 1.0}
